In [3]:
from core.utils import Tibanna
from core import ff_utils
from uuid import UUID
import ast

env = 'fourfront-webprod'
tibanna = Tibanna(env=env)
ff = ff_utils.fdn_connection(key=tibanna.ff_keys)

schema_name = {}
profiles = ff_utils.get_metadata('/profiles/', connection=ff, frame = 'raw')
for key, value in profiles.iteritems():
    schema_name[key] = value['id'].split('/')[-1][:-5]

def is_uuid(value):
    #md5 qualifies as uuid
    if '-' not in value:
        return False
    try:
        UUID(value, version=4)
        return True
    except:
        return False

def find_uuids(val):
    vals = []
    if not val:
        return []
    elif isinstance(val, basestring):
        if is_uuid(val):
            vals = [val]
        else:
            return []
    else:
        text =str(val)
        text_list = [i for i in text. split("'") if len(i) == 36]
        vals = [i for i in text_list if is_uuid(i)]
    return vals
    

In [11]:
import json
import copy
def record_object(uuid, store, item_uuids):
    #keep list of fields that only exist in frame embedded (revlinks) that you want connected
    
#     add_from_embedded = {'file_fastq':['workflow_run_inputs', 'workflow_run_outputs'],
#                          'file_processed':['workflow_run_inputs', 'workflow_run_outputs']
#                         }
    add_from_embedded = {}
   
    #find schema name, store as obj_key, create empty list if missing in store

    obj_type = ff_utils.get_metadata(uuid, connection=ff)['@type'][0]

    try:
        obj_key = schema_name[obj_type]
    except:
        #print 'CAN NOT FIND', obj_type, uuid
        return store, item_uuids
    if obj_key not in store:
        store[obj_key] = []

    add_uuids = []
    resp = ff_utils.get_metadata(uuid, connection=ff, frame = 'raw')
    
    # if resp['status'] not in ['current', 'released', 'in review by lab', 'uploaded']:
    print obj_key, uuid, resp['status']
        
    # add raw frame to store and uuid to list
    if uuid not in item_uuids:
        store[obj_key].append(resp)
        item_uuids.append(uuid)
    # this case should not happen actually
    else:
        return store, item_uuids
    
    fields_to_check = copy.deepcopy(resp)
  
    # check if any field from the embedded frame is required
    add_fields = add_from_embedded.get(obj_key)
    if add_fields:
        emb_resp = ff_utils.get_metadata(uuid, connection=ff, frame = 'embedded')
        for a_field in add_fields:
            field_val = emb_resp.get(a_field)
            if field_val:
                fields_to_check[a_field] = field_val
        
    for key, value in fields_to_check.iteritems():
        # uuid formatted fields to skip
        if key == 'uuid':
            continue
        uuid_in_val = find_uuids(value)
        
        for a_uuid in uuid_in_val:
            if a_uuid not in item_uuids:
                store, item_uuids = record_object(a_uuid, store, item_uuids)   
    return store, item_uuids
            
sets = ["9a361b5c-1278-44fe-a90e-c88d4544bdca"]    
my_exp = sets[0]
print my_exp
store_dict = {} 
item_uuids = []

store, uuids = record_object(my_exp, store_dict, item_uuids)

print(len(uuids))
print(len(list(set(uuids))))
    


# for a_type in store:
#     filename = 'Jsons/' + a_type + '.json'
#     with open(filename, 'w') as outfile:
#         json.dump(store[a_type], outfile, indent=4)




9a361b5c-1278-44fe-a90e-c88d4544bdca
experiment_set_replicate 9a361b5c-1278-44fe-a90e-c88d4544bdca released
user 900c5bf0-5b3a-4d7c-a6e6-7918faa19a77 current
user 986b362f-4eb6-4a9c-8173-3ab267307e3a current
lab 828cd4fe-ebb0-4b36-a94a-d2e3a36cc989 current
award b0b9c607-f8b4-4f02-93f4-9895b461334b current
lab 3c577664-affb-41c4-bf27-9e21c2fc1554 current
award ae6c618f-7a8c-441e-a886-e30bbbe591da current
user 83b5073a-069b-4162-9b30-6f42d5551e34 current
award 43fc2984-a074-49c4-a69f-c9d3f3eadeba current
lab 83080563-722d-4f9e-a29a-c978d33f5ff1 current
user b2ceeab8-b15b-4fdd-bcf7-49aaca674a63 current
experiment_atacseq fd0cce90-e73f-4882-899c-4a017aaa3765 released
file_fastq ba29c867-fd94-4dd3-8688-3a0c624a1e84 released
file_fastq 7565e861-24fc-41be-9562-61da889b94c1 released
quality_metric_fastqc 449c1d24-fff0-45d4-8f30-0861695b76fa released
quality_metric_fastqc 61a1b7f9-2d79-46bb-a056-e806e3839415 released
protocol 77a6a279-23aa-48bf-871c-596a10a52951 released
protocol 8a96daa8-0f91

In [12]:
## This part is only for releasing
counter = 0
for a_type in store:
    for raw_data in store[a_type]:
        if raw_data['status'] in ['deleted']: 
            print 'deleted data', a_type ,raw_data['uuid']
            continue
        if raw_data.get('contributing_labs'):
            print('cont lab')
            cont = raw_data.get('contributing_labs')
            if isinstance(cont, (list,)):
                print('already list')
                pass
            else:
                new_val = [cont]
                print('needs update') 

                patch_data1 = {'contributing_labs': new_val}
                #ff_utils.patch_metadata(patch_data1, obj_id=raw_data['uuid'] ,connection=ff)

            
            
        if raw_data['status'] not in ['current', 'released', 'released to project']:    
            counter +=1
            print a_type, raw_data['status'], raw_data['uuid']
            patch_data = {'status': "released"}
            ff_utils.patch_metadata(patch_data, obj_id=raw_data['uuid'] ,connection=ff)
print counter

cont lab
already list
cont lab
already list
cont lab
already list
cont lab
already list
cont lab
already list
cont lab
already list
cont lab
already list
cont lab
already list
cont lab
already list
cont lab
already list
cont lab
already list
cont lab
already list
cont lab
already list
cont lab
already list
cont lab
already list
cont lab
already list
0
